# Basic run

In [23]:
import pickle
from energyscope.energyscope import Energyscope
from energyscope.models import infrastructure_ch_2050, Model
from energyscope.result import postprocessing
from energyscope.plots import plot_sankey

import pandas as pd
import plotly.graph_objects as go

In [24]:
path_core_dat = 'src/energyscope/data/datasets/core/td/belgium'
path_core_mod = 'src/energyscope/data/models/core/td'

In [25]:
Model_es_core_bel = Model(mod_files=[path_core_mod+"/ESTD_model_core.mod"],
                               dat_files=[path_core_dat+"/ESTD_12TD.dat",path_core_dat+"/ESTD_data_core.dat"])

es_core_bel = Energyscope(model=Model_es_core_bel)
es_core_bel = es_core_bel.calc()

Gurobi 11.0.0:Gurobi 11.0.0: optimal solution; objective 37363.41115
40851 simplex iteration(s)
54 barrier iteration(s)


In [26]:
Model_es_bel = Model(mod_files=[path_core_mod+"/ESTD_model.mod"],
                               dat_files=[path_core_dat+"/ESTD_12TD.dat",path_core_dat+"/ESTD_data.dat"])
es_bel = Energyscope(model=Model_es_bel)
es_bel = es_bel.calc()

Gurobi 11.0.0:Gurobi 11.0.0: optimal solution; objective 48795.86549
39888 simplex iteration(s)
77 barrier iteration(s)


In [27]:
#23% TOTEX difference

es_core_bel_df = es_core_bel.variables['F']
es_bel_df = es_bel.variables['F']

# Merge the DataFrames
merged_df = es_core_bel_df.rename(columns={'F': 'F_core'}).merge(
    es_bel_df.rename(columns={'F': 'F_bel'}), 
    left_index=True, right_index=True
)

# Compute percentage variation
merged_df['Percentage_Variation'] = ((merged_df['F_core'] - merged_df['F_bel']) / merged_df['F_bel']) * 100

# Filter only percentage variations greater than or equal to 5%
filtered_df = merged_df[abs(merged_df['Percentage_Variation']) >= 5]

# Create the bar chart using Plotly Graph Objects
fig = go.Figure()

fig.add_trace(go.Bar(
    x=filtered_df.index,
    y=filtered_df['Percentage_Variation'],
    name='Percentage Variation',
    marker_color='indianred'
))

# Update layout to add legend
fig.update_layout(
    title="Percentage Variation in Installed Capacities",
    xaxis_title="Technology",
    yaxis_title="Percentage Variation [%]",
    xaxis_tickangle=-45,
    legend_title="Legend"
)

# Show the plot
fig.show()